## Import Python Library
### We have imported all the relevance Library
- Pandas
- Numpy
- gensim
- nltk - Natural Language Tool Kit
- pyLDAvis
- pickle

In [30]:
import pandas as pd
import numpy as np
import os
import re

import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')

import gensim.corpora as corpora
from nltk.corpus import stopwords
from pprint import pprint

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

import pickle 

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sushanta/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Import Data
##### The data is extracted from the www.quora.com. The data is extracted by manually from searching the relevance of UUIC in the Computer Science Department. These documents (text extracted) are stored in the excel sheet (quora_data.xls) with 4 columns as below

- URL 
- Relevance
- Question
- Answer

##### We are interested in those questions and its related answer where "Relevance" = "Yes". These questions are related only to University of Illinois Urbana Champaign (UIUC UC) Computer Science Department, where users are raising question related to know the feedback of the department. Other questions like comparision of of University and other questions, which we have marked as "Relevance" = "No".


In [4]:
# Read the Quora data from the quora_data.csv file
quora_data = pd.read_csv('quora_data.csv')

# Print all the Columns
quora_data.columns

Index(['URL', 'Relevance', 'Question', 'Answer'], dtype='object')

## Data Description & Extraction

In [5]:
#Total Number of data in the quora_data
print ("Number of rows & data in the quora_data: {}".format(quora_data.shape))

#Total Number of distinct URL for the questions
print ("Number of Distinct URL in quora_data: {}".format(len(quora_data['URL'].unique())))

#Total Number of data in the quora_data which are relevanced
quora_data_relevance = quora_data[quora_data["Relevance"] == "Yes"]
print ("Relevanced Data in quora_data (Relevance = Yes): {}".format(quora_data_relevance.shape))


#Total Number of distinct URL for the questions which are Relevanced
print ("Number of Distinct URL which are Relevance: {}".format(len(quora_data_relevance['URL'].unique())))

#Show all the Questions 
print ("-----------------------------------------------------------")
for i in np.arange(len(quora_data_relevance["Question"].unique())):
    print ("{}.{}".format(i+1,quora_data_relevance["Question"].unique()[i]))
print ("-----------------------------------------------------------")

#Show some of the answers to the questions 
for i in np.arange(5):
    print ("{}.{}".format(i+1,quora_data_relevance["Answer"].unique()[i]))
    print ("\n\n")
print ("-----------------------------------------------------------")

Number of rows & data in the quora_data: (56, 4)
Number of Distinct URL in quora_data: 12
Relevanced Data in quora_data (Relevance = Yes): (40, 4)
Number of Distinct URL which are Relevance: 9
-----------------------------------------------------------
1.What is it like to study computer science at UIUC?
2.How hard is it to get into computer science at UIUC?
3.How is life as a computer science student at UIUC's college of engineering?
4.How is the University of Illinois, Chicago for an MS in CS?
5.Why UIUC CS is so highly ranked when it is relatively easier to get into it as compared with other top CS schools?
6.How's the University of Illinois? How difficult is it to get into its CS program?
7.What's the reputation of the University of Illinois at Chicago (UIC) for CS?
8.How good is the MS in Computer Science and Engineering program of the University of Illinois at Chicago, considering the quality of education, job & internship scene and part time jobs?
9.According to the U.S. News an

## Data Cleaning
#### Following Data Cleaning are done as part of the data cleaning
- remove the special character and HTML character (".", "!", "?")
- lower the text data

In [6]:
# Remove the special characters (., !, ?, [, ])
quora_data_relevance['Answer_processed'] = quora_data_relevance['Answer'].map(lambda x: re.sub('[,\.!?]', '', x))

# Convert to the lowercase
quora_data_relevance['Answer_processed'] = quora_data_relevance['Answer_processed'].map(lambda x: x.lower())


In [9]:
stopp_words = stopwords.words('english')

#Below are some of the words which are extended other than included of the english stop words
stopp_words.extend(['uiuc','cs','uic','university','graduate',"illinois",'school',\
                       'also','know','like','get','department','take','many','would',\
                       'chicago','since','taking','go','really','one','may','even',\
                       'lot',"computer","science"])

def convert_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

text_data = quora_data_relevance.Answer_processed.values.tolist() # Convert the text into list
final_data = list(convert_words(text_data))

#Removing the stop words
final_data = [[word for word in simple_preprocess(str(doc)) if word not in stopp_words] for doc in final_data]


## Create the Model and fit into LDA to generate the the Top Topics

In [15]:
# Create Dictionary
id_to_word = corpora.Dictionary(final_data)

corpus = [id_as_word.doc2bow(text) for text in final_data]

# Number of Topics
num_topics = 5

# Create the LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,id2word=id_to_word,num_topics=num_topics)
doc_lda = lda_model[corpus]

In [27]:
#Create the folder & file location where the file needs to be dumped
folder_file_location = 'ldavis_'+str(num_topics)
ldavis_file_path = os.path.join(folder_file_location)

#Create the ldavis model and dump the model in the specified folder
ldavis_model = gensimvis.prepare(lda_model, corpus, id_to_word)
with open(ldavis_file_path, 'wb') as f:
    pickle.dump(ldavis_model, f)

#Save the Model in the HTML form
pyLDAvis.save_html(ldavis_model, folder_file_location +'.html')

ldavis_model


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.004100 -0.002801       1        1  27.573788
1      0.013317  0.001738       2        1  25.833278
2      0.023238 -0.031332       3        1  19.450284
0     -0.052896 -0.004911       4        1  16.304885
3      0.012241  0.037306       5        1  10.837765, topic_info=             Term       Freq      Total Category  logprob  loglift
498   scholarship   5.000000   5.000000  Default  30.0000  30.0000
290         first   6.000000   6.000000  Default  29.0000  29.0000
309       applied   4.000000   4.000000  Default  28.0000  28.0000
518        option   3.000000   3.000000  Default  27.0000  27.0000
142        better  12.000000  12.000000  Default  26.0000  26.0000
..            ...        ...        ...      ...      ...      ...
23        courses   0.936445   8.592057   Topic5  -5.7852   0.0056
600  universities   0.838360   5.458181   Topic5  -5.8958   0.3487
607       caltech   0.971613  12.913487   Topic5  -5.7483  -0.3649
146       college   0.864557  10.700478   Topic5  -5.8651  -0.2937
433      programs   0.848546  11.939615   Topic5  -5.8837  -0.4220

[337 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
207       1  0.232708    able
207       2  0.465416    able
207       3  0.232708    able
207       4  0.232708    able
520       1  0.496226  abroad
...     ...       ...     ...
664       1  0.225743     yes
664       2  0.451487     yes
664       4  0.225743     yes
604       2  0.499639  yocket
604       5  0.499639  yocket

[537 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 2, 3, 1, 4])

/opt/anaconda3/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/opt/anaconda3/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/opt/anaconda3/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/opt/anaconda3/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/opt/anaconda3/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in fav